In [1]:
import time
from pandas import DataFrame
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import *
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import LinearSVC, SVC
from data_initialization import *
from sklearn.feature_selection import RFECV, RFE
from sklearn.naive_bayes import GaussianNB
results = dict()

for kernel in ["linear", "poly", "rbf", "sigmoid"]:
    for C in [0.25, 0.5, 1, 2]:
        for train_size in [0.99]:
            X, y = get_data("c", "phone", "accel")
            X, _, y, _ = train_test_split(X, y, train_size=train_size)
            for n_estimators in [5, 10, 20, 50]:
                print("n estimators: %d" % n_estimators)
                print("kernel: %s" % kernel)
                print("C: %02f" % C)

                clf = OneVsRestClassifier(BaggingClassifier(SVC(kernel=kernel, C=C), max_samples=1.0 / n_estimators, n_estimators=n_estimators))

                start = time.time()
                scores = cross_validate(clf, X, y, cv=5, n_jobs=-1)
                end = time.time()
                results[(kernel, C, n_estimators)] = scores

                print("fit time: %0fs" % np.mean(scores['fit_time']), end=" +/- ")
                print("%0fs" % np.std(scores['fit_time']))
                print("score time: %0fs" % np.mean(scores['score_time']), end=" +/- ")
                print("%0fs" % np.std(scores['score_time']))
                print("score: %0f%%" % (np.mean(scores['test_score']*100)), end=" +/- ")
                print("%0f%%" % (np.std(scores['test_score']*100)))
                print("actual time: %f" % (end-start))
                print("\n===================\n")

print(results)

n estimators: 5
kernel: linear
C: 0.250000
fit time: 25.829349s +/- 0.440969s
score time: 8.786721s +/- 0.072747s
score: 36.212394% +/- 0.554688%
actual time: 36.799132


n estimators: 10
kernel: linear
C: 0.250000
fit time: 16.404571s +/- 0.194979s
score time: 9.815648s +/- 0.104286s
score: 36.597652% +/- 0.662163%
actual time: 27.470125


n estimators: 20
kernel: linear
C: 0.250000
fit time: 17.344979s +/- 0.282745s
score time: 11.448499s +/- 0.166010s
score: 36.475083% +/- 0.698072%
actual time: 30.235807


n estimators: 50
kernel: linear
C: 0.250000
fit time: 35.267588s +/- 0.625811s
score time: 13.681407s +/- 0.083232s
score: 35.065402% +/- 0.774125%
actual time: 50.188674


n estimators: 5
kernel: linear
C: 0.500000
fit time: 28.132398s +/- 0.271239s
score time: 9.229350s +/- 0.063333s
score: 36.194861% +/- 0.929044%
actual time: 37.900648


n estimators: 10
kernel: linear
C: 0.500000
fit time: 18.218307s +/- 0.335594s
score time: 10.171422s +/- 0.056661s
score: 37.171199% +/- 0.

In [12]:
for k, v in results.items():
    v["fit_time"] = np.mean(v["fit_time"])
    v["score_time"] = np.mean(v["score_time"])
    v["test_score"] = np.mean(v["test_score"])

In [16]:
df = {
    "fit_time":[],
    "score_time":[],
    "test_score":[],
    "kernel":[],
    "C":[],
    "n_estimators":[]
}
for kernel in ["linear", "poly", "rbf", "sigmoid"]:
    for C in [0.25, 0.5, 1, 2]:
            for n_estimators in [5, 10, 20, 50]:
                df['kernel'].append(kernel)
                df['C'].append(C)
                df['n_estimators'].append(n_estimators)
                values = results[(kernel, C, n_estimators)]
                df['fit_time'].append(values['fit_time'])
                df['score_time'].append(values['score_time'])
                df['test_score'].append(values['test_score'])

df = DataFrame(df)
for column in df.columns:
    print("sorted by %s:" % column)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df.sort_values(by=[column]))
    print("\n===============\n")

sorted by fit_time:
     fit_time  score_time  test_score   kernel     C  n_estimators
49  11.312948   11.629312    0.046929  sigmoid  0.25            10
61  11.740035   11.732814    0.047410  sigmoid  2.00            10
53  11.962235   11.755219    0.046185  sigmoid  0.50            10
57  12.035427   11.866494    0.048286  sigmoid  1.00            10
33  13.641050   39.402557    0.525106      rbf  0.25            10
17  13.834921    8.513443    0.514643     poly  0.25            10
37  13.932746   38.836132    0.537583      rbf  0.50            10
21  14.021837    8.506864    0.532242     poly  0.50            10
41  14.169932   39.440548    0.542310      rbf  1.00            10
45  14.221361   38.994117    0.548702      rbf  2.00            10
25  14.291785    8.434880    0.556363     poly  1.00            10
29  14.946821    8.388563    0.576238     poly  2.00            10
50  15.059539   13.052829    0.049818  sigmoid  0.25            20
62  15.100407   13.034468    0.041851  sig